In [4]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo-0613"): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

def find_code(Text):
    start_prompt = f'Extract the code snippet hidden in \n\n\n{Text}\n\n\n and if there is no code snippet, say no code present:'
    
    end_prompt = '\n\nExtracted Code: '
    
    prompt = start_prompt + end_prompt

    code = get_completion(prompt)

    return (code)

In [5]:
Text = 'In software development, commenting code is a common and recommended practice. Comments are used to explain what the code does, why it does it, and how it does it. They make the code easier to understand and debug, especially for other developers who might be working on the same project. In C++, comments can be written as either single-line or multi-line comments. Single-line comments are written using two forward slashes (//), and everything to the right of the slashes is considered a comment. Multi-line comments are written using a forward slash and an asterisk (/*) to begin the comment, and an asterisk and a forward slash (*/) to end the comment.'

In [6]:
find_code(Text)

'no code present'

# Loading DataSet

In [1]:
import pandas as pd
DataSet = pd.read_json(r"C:\Users\Asus\Downloads\Participants_Dataset_SHPACT\datafinal.json")
df_train = DataSet[(DataSet['ContainsCode']==True)].reset_index(drop=True)
df_test = DataSet[(DataSet['ContainsCode']=='')].reset_index(drop=True)
df_without_code = DataSet[(DataSet['ContainsCode']==False)].reset_index()
Training_data = DataSet[(DataSet['ContainsCode']!='')].reset_index()
Testing_data = DataSet[(DataSet['ContainsCode']!=False)].reset_index()
df_train.shape,df_test.shape,df_without_code.shape,Training_data.shape,Testing_data.shape

((532, 4), (531, 4), (845, 5), (1377, 5), (1063, 5))

In [6]:
df_test['CodeList'] = df_test['Text'].apply(lambda text:find_code(text))

In [7]:
# df.to_csv("Shell_comp_All_GPT_work2.csv",index=False)
# df_new.to_csv("Shell_comp_All_GPT_work.csv",index=False)

In [25]:
# pd.read_csv("Shell_comp_All_GPT_work.csv")#[1040:1065]

In [27]:
# pd.read_csv("Shell_comp_All_GPT_work.csv").isnull().sum()

In [28]:
# import numpy as np
# np.where(pd.read_csv("Shell_comp_All_GPT_work.csv")['CodeList'].isnull()==True)

In [26]:
df_test.isnull().sum()

In [17]:
(df_test['CodeList']=='').sum()

0

In [19]:

for ind,code_str in enumerate(df_test['CodeList']):
    if 'no code pres' in code_str.lower():
        df_test['CodeList'][ind] = (df_test['CodeList'][ind].replace(code_str,' '))
        

In [404]:
count = 0
for ind,code_str in enumerate(df_test['CodeList']):
    if 'no code pres' in code_str.lower():
        count+=1
count

0

In [24]:
df_res = df_test.merge(Training_data,'outer').sort_values(by = 'ID').reset_index(drop=True)[['ID','Text','ContainsCode','CodeList']]

In [457]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
s1 = df_res["CodeList"]
t1 = mlb.fit_transform(s1)

from sklearn.preprocessing import MultiLabelBinarizer
mlb2 = MultiLabelBinarizer()
s2 = DataSet["CodeList"]
t2 = mlb2.fit_transform(s2)

In [21]:
print("Classes Present for results:")
print(mlb.classes_)
print("Classes Present for DataSet:")
print(mlb2.classes_)
print("Classes size for results: ",len(mlb.classes_))
print("Classes size for DataSet: ",len(mlb2.classes_))

In [23]:
for ind,string in enumerate(s1):
    if ('\n' in string):
         s1[ind] = (s1[ind].replace('\n','\\n'))
    if '\t' in string:
        s1[ind] = (s1[ind].replace('\t','\\t'))        

In [108]:
t1 = mlb.fit_transform(s1)
submission = pd.DataFrame(t1)
submission.columns

RangeIndex(start=0, stop=93, step=1)

In [109]:
submission.to_csv("submission.csv",index=False)